In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
def rep_str(value):
    replace_chars = {'': [")", '.)', ',', '/', "ikom",
                         'u\n', "'", ".0", 
                          "years", ' years', "dead",
                          "_years", "*"],
                    '_': ['- ',' (', ' ', '-', "__"]
                    }

    for new, old in replace_chars.items():
        for c in old:
            if type(value) == str:
                value = value.replace(c, new).rstrip()
            else:
                continue

    return value

# mapping missing or mistyped answers
with open('col_answers_mapping.json', 'r') as f:
    col_answers_map = json.load(f)
    
answers = col_answers_map['col_answers_map']
n_col = len(answers)
    
c_map = {"col_encoding":[]}
col_enc = {}
for i in range(0, n_col):
    col = answers[i]['new_name']
    mapping = answers[i]['mapping']
    result = {"col_name":col}
    vals = list(mapping.values())
    new_vals = [rep_str(v.lower()) for v in vals]
    col_mapping = dict()
    rng = len(new_vals)
    if "yes" in new_vals and "no" in new_vals:
            col_mapping = {'no': 0, 'yes':1, 
                           'option 1': 0, 'option 2': 1}

    elif col == "school_performance_change":
        col_mapping = {
            "yes_its_gotten_worse": -1,
            'no_there_has_not_been_a_noticeable_change': 0,
            'yes_overall_school_performance_is_better': 1}

    elif any(map(col.__contains__, ["_change", "_improvement"])):

        cats = new_vals
        neutral = ['no', 'same', 'stay', 'not']
        down = ['lost', 'decrease','decreased','decreases', 'worse',]
        up = ['add', 'better', 'increase', 
                'increased','increases', 'greater']
        for cat in cats:
            cat = str(cat)
            if cat != 'nan':
                if cat == 'no' or any(map(cat.__contains__, neutral)):
                    col_mapping[cat] = 0
                elif any(map(cat.__contains__, down)):
                    col_mapping[cat] = -1
                elif cat == 'yes' or any(map(cat.__contains__, up)):
                    col_mapping[cat] = 1
    else:
        rng = len(new_vals)
        for idx, val in enumerate(new_vals):
            col_mapping[val] = idx+1
            
    result["col_enc_map"] = col_mapping
    c_map["col_encoding"].append(result)


# Serializing json
json_object = json.dumps(c_map, indent=4)
 
# Writing to sample.json
with open("col_encoding.json", "w") as outfile:
    outfile.write(json_object)